In [1]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.datasets import imdb 
from sklearn.metrics import classification_report

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
lm = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics, svm

In [5]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
n_unique_words = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=n_unique_words)

In [8]:
maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
x_train.shape

(25000, 200)

In [6]:
#pure bilstm
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [7]:
#model.fit(x_train, y_train, validation_split = 0.2,epochs = 12, verbose = True,batch_size = 64)

In [22]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [38]:
inp = Input(shape=(200, ))
x = Embedding(10000, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_1)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 200)]             0         
                                                                 
 embedding_12 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 12, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 bidirectional_10 (Bidirecti  (None, 12, 128)          49664     
 onal)                                                           
                                                           

In [39]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es

In [40]:
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")

In [41]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#file_path = 'model.hdf5'
#ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#early = EarlyStopping(monitor="val_loss", mode="min", patience=1)

In [42]:
model.fit(x_train, y_train, batch_size=256, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
79/79 [==============================] - 19s 179ms/step - loss: 0.6889 - accuracy: 0.5337 - val_loss: 0.6628 - val_accuracy: 0.5972
Epoch 2/20
79/79 [==============================] - 14s 173ms/step - loss: 0.6039 - accuracy: 0.6755 - val_loss: 0.5384 - val_accuracy: 0.7326
Epoch 3/20
79/79 [==============================] - 14s 174ms/step - loss: 0.5147 - accuracy: 0.7505 - val_loss: 0.5073 - val_accuracy: 0.7514
Epoch 4/20
79/79 [==============================] - 14s 176ms/step - loss: 0.4641 - accuracy: 0.7862 - val_loss: 0.4748 - val_accuracy: 0.7680
Epoch 5/20
79/79 [==============================] - 14s 181ms/step - loss: 0.4103 - accuracy: 0.8184 - val_loss: 0.4901 - val_accuracy: 0.7654
Epoch 6/20
79/79 [==============================] - 14s 183ms/step - loss: 0.3811 - accuracy: 0.8316 - val_loss: 0.4464 - val_accuracy: 0.7908
Epoch 7/20
79/79 [==============================] - 15s 184ms/step - loss: 0.3389 - accuracy: 0.8550 - val_loss: 0.4554 - val_accuracy: 0.7880

In [43]:
y_pred = model.predict(x_test)

782/782 [==============================] - 7s 9ms/step


In [44]:
y_pred.shape

(25000, 1)

In [45]:
y_test.shape

(25000,)

In [46]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [47]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77     12500
           1       0.75      0.87      0.80     12500

    accuracy                           0.79     25000
   macro avg       0.80      0.79      0.79     25000
weighted avg       0.80      0.79      0.79     25000



In [48]:
new_df = pd.read_csv('Data_set.txt',delimiter = "\t",encoding='latin-1')

In [49]:
new_df.head()

,Rating,Description
0,0,what's surprising about this traditional thril...
1,0,the essential problem in orange county is that...
2,1,a devastating indictment of unbridled greed an...
3,1,a breezy romantic comedy that has the punch of...
4,1,drumline ably captures the complicated relatio...


In [50]:
y = new_df['Rating']
X = new_df['Description']
def clean_data(data):
    data = data.replace("\n", "")
    data = re.sub(r'\W', ' ', data)
    data = re.sub(r'\s+[a-zA-Z]\s+', ' ', data)
    data = re.sub(r'\s+', ' ', data, flags=re.I)
    data = data.lower().split(" ")
    data = [word for word in data if word not in set(stopwords.words("english"))]                                            
    data = ' '.join([str(elem) for elem in data])
    return data

In [51]:
incrementer=0
for i in X:
    i = clean_data(i)
    X[incrementer]=i
    incrementer+=1

C:\Users\HP\AppData\Local\Temp\ipykernel_20368\2672807160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[incrementer]=i


In [52]:
incrementer=0
for i in X:
    listw = nltk.word_tokenize(i)
    lemmatized_string = ' '.join([lm.lemmatize(words) for words in listw])
    X[incrementer]=lemmatized_string
    incrementer+=1

C:\Users\HP\AppData\Local\Temp\ipykernel_20368\3635973817.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[incrementer]=lemmatized_string


In [54]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

In [55]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [56]:
X_train[0].shape

(1, 14406)

In [57]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1 
print(vocab_size)

14436


In [58]:
maxlen = 500

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

X_train.shape

(7996, 500)

In [59]:
vocab_size

14436

In [60]:
inp = Input(shape=(500, ))
x = Embedding(14436, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_1)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 500)]             0         
                                                                 
 embedding_13 (Embedding)    (None, 500, 128)          1847808   
                                                                 
 conv1d_1 (Conv1D)           (None, 492, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 30, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 30, 32)            0         
                                                                 
 bidirectional_11 (Bidirecti  (None, 30, 128)          49664     
 onal)                                                           
                                                           

In [61]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
100/100 [==============================] - 17s 123ms/step - loss: 0.6904 - accuracy: 0.5460 - val_loss: 0.6896 - val_accuracy: 0.5506
Epoch 2/20
100/100 [==============================] - 12s 119ms/step - loss: 0.6898 - accuracy: 0.5458 - val_loss: 0.6880 - val_accuracy: 0.5506
Epoch 3/20
100/100 [==============================] - 12s 121ms/step - loss: 0.6894 - accuracy: 0.5458 - val_loss: 0.6884 - val_accuracy: 0.5506
Epoch 4/20
100/100 [==============================] - 12s 124ms/step - loss: 0.6887 - accuracy: 0.5458 - val_loss: 0.6877 - val_accuracy: 0.5506
Epoch 5/20
100/100 [==============================] - 12s 124ms/step - loss: 0.6843 - accuracy: 0.5442 - val_loss: 0.6842 - val_accuracy: 0.5506
Epoch 6/20
100/100 [==============================] - 13s 125ms/step - loss: 0.6524 - accuracy: 0.6188 - val_loss: 0.6882 - val_accuracy: 0.5556
Epoch 7/20
100/100 [==============================] - 13s 128ms/step - loss: 0.6297 - accuracy: 0.6520 - val_loss: 0.6921 - val_ac

In [62]:
y_pred = model.predict(X_test)

84/84 [==============================] - 2s 17ms/step


In [63]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [64]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.59      0.81      0.68      1492
           1       0.54      0.29      0.37      1174

    accuracy                           0.58      2666
   macro avg       0.56      0.55      0.53      2666
weighted avg       0.57      0.58      0.55      2666

